In [ ]:
import pandas as pd
import numpy as np
import MeCab
import mojimoji
import gensim
m = MeCab.Tagger('-Ochasen')
m.parse('')

# 製品名を削除

In [ ]:
def extract_brackets(text):
    if text.find('(') != -1:
        return text[:text.find('('):]
    return text

def replace_summary(row):
    item = extract_brackets(mojimoji.zen_to_han(row['item'], kana=False))
    return row['accident_summary'].replace(item, ' ').replace('使用中', ' ').replace('使用', ' ')

df['replaced_as'] = df.replace('(【|【)', '(', regex=True).replace('(】|】)', ')', regex=True).apply(lambda row: replace_summary(row), axis=1)

----
# パースするところ

In [ ]:
def read_stopword():
    with open('./wmd/stopwords.txt', 'r') as f:
        stop_words = f.read().splitlines()
    return stop_words

stop_words = read_stopword()

-----
# word2vecのベクトルに使う単語を抽出するところ

In [ ]:
df = pd.read_csv('../data//scdv-ver_input_injured.csv')

In [ ]:
# 最後の名詞を取る
def extract_noun(sentence):
    for chunk in m.parse(sentence.split('。')[0]).splitlines()[-2::-1]:
        feature = chunk.split('\t')
        if '名詞' in feature[3]:
            return feature[2]
    return ''

def parse_sentence(sentence):
    nouns = ''
    for chunk in m.parse(sentence.split('。')[0]).splitlines()[:-1]:
        feature = chunk.split('\t')
        if '名詞-サ変接続' in feature[3] or '負う' in feature[2]:
            nouns += feature[2] + ','
    return nouns[:-1]

def delte_double_words(row):
    summary = ''
    for r in list(set(row.split(','))):
        summary += r + ','
    return summary[:-1]

In [ ]:
df.head()

In [ ]:
df['replaced_as'] = df['replaced_as'].replace('[0-9]+', '0', regex=True).replace('(０|１|２|３|４|５|６|７|８|９)+', '0', regex=True)
noun = df['replaced_as'].apply(lambda row: extract_noun(mojimoji.zen_to_han(row, kana=False)))
df['parsed_summary'] = df['replaced_as'].apply(lambda row: parse_sentence(mojimoji.zen_to_han(row, kana=False), stop_words, other_flag)) + noun
df['parsed_summary'] = df['parsed_summary'].apply(lambda row: delte_double_words(row))

In [ ]:
df.to_csv('../../data/word2vec-ver_input_injured.csv', index=False)

-----
# 怪我を負ったレポートに対してword2vecで平均ベクトルを作成

In [ ]:
df = pd.read_csv('../../data/word2vec-ver_input_injured.csv')

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('../entity_vector/entity_vector.model.bin', binary=True)

In [ ]:
def calc_avg_vector(words, model, vec_size):
    avg_vec = np.zeros(vec_size, )
    for word in words:
        try:
            avg_vec += model[word]
        except:
            avg_vec += np.zeros(vec_size, )
    return avg_vec / len(words)

vec_size = model.vector_size
avg_vec = df['parsed_summary'].apply(lambda row: calc_avg_vector(row.split(','), model, vec_size))

In [ ]:
# concat dataframe
col_list = ['d'+str(i) for i in range(avg_vec[:1][0].shape[0])]
out_df = pd.DataFrame(columns=col_list)
for sv in avg_vec:
    out_df = out_df.append(pd.Series(sv, index=out_df.columns), ignore_index=True)

out_df.to_csv('../output/word2vec_out.csv', index=False)